The decision variables are
$$
x_{ijk} = \begin{cases}
    1, & \text{ if element } (i,j) \text{ of the } n\times n \text{ Sudoku matrix contains the integer } k \\
    0, & \text{ otherwise.}
\end{cases}
$$

A binary integer programming formulation is
$$
\begin{align*}
    \text{ min } & \textbf{0}^T\textbf{x} \\
    \text{ s.t } & \sum_{i=1}^n x_{ijk} = 1, \quad j = 1:n, k = 1:n \quad \text{ (only one } k \text{ in each column)} \\
    & \sum_{j=1}^n x_{ijk} = 1, \quad i = 1:n, k = 1:n \quad \text{ (only one } k \text{ in each row)} \\
    & \sum_{j = mq - m + 1}^{mq} \sum_{i = mp - m + 1}^{mp} x_{ijk} = 1, \quad k = 1:n, p = 1:m, q = 1:m \quad \text{ (only one } k \text{ in each submatrix)} \\
    & \sum_{k = 1}^n x_{ijk} = 1 \quad i = 1:n, j = 1:n \quad \text{ ( every position in matrix must be filled)} \\
    & x_{ijk} = 1 \quad \forall (i,j,k) \in G \quad \text{ (given elements } G \text{ in matrix are set ``on")} \\
    & x_{ijk} \in \{0,1\} \\
    & i \in \{1,2,...,9\} \\
    & j \in \{1,2,...,9\} \\
    & k \in \{1,2,...,9\} \\
    & p \in \{1,2,3\} \\
    & q \in \{1,2,3\} 
\end{align*}
$$
Where $m = \sqrt n$

In [5]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

# Set up size of the Sudoku matrix
size = 9
boxsize = np.sqrt(size)

# Create model
model = gp.Model('Sudoku')

# Create decision variables
x_ijk = model.addVars(size, size, size, vtype=GRB.BINARY, 'x_ijk')

# Add contraints for the rules of Sudoku
# Ensure only one k in each column
model.addConstrs((x_ijk.sum('*',j,k) == 1 for j in range(size) for k in range (size)), 'c1')

# Ensure only one k in each row
model.addConstrs((x_ijk.sum(i,'*',k) == 1 for i in range(size) for k in range (size)), 'c2')

# Ensure only one k in each submatrix
for q in range(boxsize):
    for p in range(boxsize):
        model.addConstrs((x_ijk(i,j,k) == 1 for i in range(boxsize*p - boxsize + 1, boxsize*p) for j in range(boxsize*q - boxsize + 1, boxsize*q) for k in range (size)), 'c3')

# Exactly one k in every cell
model.addConstrs((x_ijk.sum(i,j,'*') == 1 for i in range(size) for j in range (size)), 'c4')

SyntaxError: positional argument follows keyword argument (3720837534.py, line 13)